In [189]:
import pandas as pd
import numpy as np
import json
import csv
import os
import glob

In [190]:
application = pd.read_csv("/Users/SamuelLP/Desktop/git/Challenge_Nexialog/data/application_train_vf.csv", sep=",", index_col=0)
bureau = pd.read_csv("/Users/SamuelLP/Desktop/git/Challenge_Nexialog/data/bureau.csv", sep=",")
POS_CASH_balance = pd.read_csv("/Users/SamuelLP/Desktop/git/Challenge_Nexialog/data/POS_CASH_balance.csv", sep=",")
credit_card_balance = pd.read_csv("/Users/SamuelLP/Desktop/git/Challenge_Nexialog/data/credit_card_balance.csv", sep=",")
previous_application = pd.read_csv("/Users/SamuelLP/Desktop/git/Challenge_Nexialog/data/previous_application.csv", sep=",")
installments_payments = pd.read_csv("/Users/SamuelLP/Desktop/git/Challenge_Nexialog/data/installments_payments.csv", sep=",")

cols_from_application = ["SK_ID_CURR", 'TARGET',"date_mensuelle" ,"NAME_CONTRACT_TYPE", "AMT_INCOME_TOTAL", "AMT_CREDIT",
                         "AMT_ANNUITY", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE",
                         "NAME_FAMILY_STATUS",'NAME_HOUSING_TYPE', 'DAYS_EMPLOYED',
                         "ORGANIZATION_TYPE", "EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]

cols_from_bureau = ["SK_ID_CURR","AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT",
                    "AMT_CREDIT_SUM_OVERDUE", "CREDIT_TYPE", "AMT_ANNUITY"]

cols_from_POS_CASH_balance = ["SK_ID_CURR", "CNT_INSTALMENT_FUTURE"]

cols_from_credit_card_balance = ["SK_ID_CURR","AMT_BALANCE", "AMT_DRAWINGS_CURRENT", "AMT_PAYMENT_CURRENT"]

cols_from_previous_application = ['SK_ID_CURR',"AMT_APPLICATION", "AMT_CREDIT", "RATE_INTEREST_PRIMARY",
                                  "NAME_CLIENT_TYPE", "NFLAG_INSURED_ON_APPROVAL"]

cols_from_installments_payments = ['SK_ID_CURR',"AMT_PAYMENT"]

In [191]:
application = application[cols_from_application]
bureau = bureau[cols_from_bureau]
POS_CASH_balance = POS_CASH_balance[cols_from_POS_CASH_balance]
credit_card_balance = credit_card_balance[cols_from_credit_card_balance]
previous_application = previous_application[cols_from_previous_application]
installments_payments = installments_payments[cols_from_installments_payments]

In [192]:
for cols in application.select_dtypes(include=['object']).columns:
    print(cols)
    print(application[cols].isna().sum()/application.shape[0]*100)

date_mensuelle
0.0
NAME_CONTRACT_TYPE
0.0
NAME_INCOME_TYPE
0.0
NAME_EDUCATION_TYPE
0.0
NAME_FAMILY_STATUS
0.0
NAME_HOUSING_TYPE
0.0
ORGANIZATION_TYPE
0.0


In [193]:
for cols in application.select_dtypes(include=np.number).columns:

    if application[cols].isna().sum()/application.shape[0]*100 > 35: application.drop(cols, axis=1, inplace=True)
    else : application.fillna({cols: application[cols].median()}, inplace=True)

## Bureau

In [194]:
# bureau
for cols in bureau.select_dtypes(include=np.number).columns:

    if bureau[cols].isna().sum()/bureau.shape[0]*100 > 35: bureau.drop(cols, axis=1, inplace=True)
    else : bureau.fillna({cols: bureau[cols].median()}, inplace=True)

In [195]:
# bureau
for cols in bureau.select_dtypes(include="object").columns:

    if bureau[cols].isna().sum()/bureau.shape[0]*100 > 35: bureau.drop(cols, axis=1, inplace=True)
    else : bureau.fillna({cols: bureau[cols].mode()}, inplace=True)

In [196]:
bureau.head()

,SK_ID_CURR,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE
0,215354,91323.0,0.0,0.0,0.0,Consumer credit
1,215354,225000.0,171342.0,0.0,0.0,Credit card
2,215354,464323.5,0.0,0.0,0.0,Consumer credit
3,215354,90000.0,0.0,0.0,0.0,Credit card
4,215354,2700000.0,0.0,0.0,0.0,Consumer credit


In [197]:
bureau_temp1 = bureau.groupby('SK_ID_CURR')[['AMT_CREDIT_SUM', "AMT_CREDIT_SUM_DEBT", 'AMT_CREDIT_SUM_LIMIT', "AMT_CREDIT_SUM_OVERDUE"]].mean()

In [198]:
compte_par_id = bureau.groupby(['SK_ID_CURR', 'CREDIT_TYPE']).size().reset_index(name='Nombre_Occurrences')

bureau_sorted = compte_par_id.sort_values(by=['SK_ID_CURR', 'Nombre_Occurrences', 'CREDIT_TYPE'], ascending=[True, False, True])

bureau_temp2 = bureau_sorted.drop_duplicates(subset=['SK_ID_CURR'], keep='first')

bureau = bureau_temp1.merge(bureau_temp2, on='SK_ID_CURR')
bureau.drop("Nombre_Occurrences", axis=1, inplace=True)
bureau.head()

,SK_ID_CURR,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE
0,100001,207623.571429,85240.928571,0.000000,0.0,Consumer credit
1,100002,108131.945625,30722.625000,3998.570625,0.0,Consumer credit
2,100003,254350.125000,0.000000,202500.000000,0.0,Consumer credit
3,100004,94518.900000,0.000000,0.000000,0.0,Consumer credit
4,100005,219042.000000,189469.500000,0.000000,0.0,Consumer credit


In [199]:
bureau['CREDIT_TYPE'].unique()

array(['Consumer credit', 'Car loan', 'Credit card', 'Microloan',
       'Mortgage', 'Another type of loan',
       'Loan for business development',
       'Loan for working capital replenishment', 'Unknown type of loan',
       'Real estate loan', 'Cash loan (non-earmarked)',
       'Loan for the purchase of equipment'], dtype=object)

## POS_CASH_balance

In [200]:
POS_CASH_balance

,SK_ID_CURR,CNT_INSTALMENT_FUTURE
0,182943,45.0
1,367990,35.0
2,397406,9.0
3,269225,42.0
4,334279,35.0
...,...,...
10001353,226558,0.0
10001354,141565,0.0
10001355,315695,0.0
10001356,450255,0.0


In [201]:
POS_CASH_balance.fillna({"CNT_INSTALMENT_FUTURE": POS_CASH_balance["CNT_INSTALMENT_FUTURE"].median()}, inplace=True)

In [202]:
POS_CASH_balance = POS_CASH_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].mean().reset_index(name='CNT_INSTALMENT_FUTURE')

POS_CASH_balance



,SK_ID_CURR,CNT_INSTALMENT_FUTURE
0,100001,1.444444
1,100002,15.000000
2,100003,5.785714
3,100004,2.250000
4,100005,7.181818
...,...,...
337247,456251,4.666667
337248,456252,3.000000
337249,456253,2.000000
337250,456254,10.350000


## credit_card_balance

In [203]:
for cols in credit_card_balance.select_dtypes(include=np.number).columns:

    if credit_card_balance[cols].isna().sum()/credit_card_balance.shape[0]*100 > 35: credit_card_balance.drop(cols, axis=1, inplace=True)
    else : credit_card_balance.fillna({cols: credit_card_balance[cols].median()}, inplace=True)

In [204]:
for cols in credit_card_balance.select_dtypes(include=["object"]).columns:

    if credit_card_balance[cols].isna().sum()/credit_card_balance.shape[0]*100 > 35: credit_card_balance.drop(cols, axis=1, inplace=True)
    else : credit_card_balance.fillna({cols: credit_card_balance[cols].mode()}, inplace=True)

In [205]:

credit_card_balance.columns

Index(['SK_ID_CURR', 'AMT_BALANCE', 'AMT_DRAWINGS_CURRENT',
       'AMT_PAYMENT_CURRENT'],
      dtype='object')

In [206]:
credit_card_balance = credit_card_balance.groupby('SK_ID_CURR')[['SK_ID_CURR', 'AMT_BALANCE', 'AMT_DRAWINGS_CURRENT',
       'AMT_PAYMENT_CURRENT']].mean()

In [207]:
credit_card_balance

,SK_ID_CURR,AMT_BALANCE,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT
SK_ID_CURR,,,,
100006,100006.0,0.000000,0.000000,2702.700000
100011,100011.0,54482.111149,2432.432432,4843.064189
100013,100013.0,18159.919219,5953.125000,7168.346250
100021,100021.0,0.000000,0.000000,2702.700000
100023,100023.0,0.000000,0.000000,2702.700000
...,...,...,...,...
456244,456244.0,131834.730732,26842.388049,32720.544878
456246,456246.0,13136.731875,15199.256250,16768.828125
456247,456247.0,23216.396211,2149.506474,4883.755263


## previous_application

In [208]:
for cols in previous_application.select_dtypes(include=np.number).columns:

    if previous_application[cols].isna().sum()/previous_application.shape[0]*100 > 35: previous_application.drop(cols, axis=1, inplace=True)
    else : previous_application.fillna({cols: previous_application[cols].median()}, inplace=True)

In [209]:
for cols in previous_application.select_dtypes(include="object").columns:

    if previous_application[cols].isna().sum()/previous_application.shape[0]*100 > 35: previous_application.drop(cols, axis=1, inplace=True)
    else : previous_application.fillna({cols: previous_application[cols].mode()}, inplace=True)

In [210]:
previous_application.head()

,SK_ID_CURR,AMT_APPLICATION,AMT_CREDIT,NAME_CLIENT_TYPE
0,271877,17145.0,17145.0,Repeater
1,108129,607500.0,679671.0,Repeater
2,122040,112500.0,136444.5,Repeater
3,176158,450000.0,470790.0,Repeater
4,202054,337500.0,404055.0,Repeater


In [211]:
previous_application_temp1 = previous_application.groupby('SK_ID_CURR')[['AMT_APPLICATION', "AMT_CREDIT"]].mean()

In [212]:
compte_par_id = previous_application.groupby(['SK_ID_CURR', 'NAME_CLIENT_TYPE']).size().reset_index(name='Nombre_Occurrences')


In [213]:

previous_application_sorted = compte_par_id.sort_values(by=['SK_ID_CURR', 'Nombre_Occurrences', 'NAME_CLIENT_TYPE'], ascending=[True, False, True])

previous_application_temp2 = previous_application_sorted.drop_duplicates(subset=['SK_ID_CURR'], keep='first')

previous_application = previous_application_temp1.merge(previous_application_temp2, on='SK_ID_CURR')
previous_application.drop("Nombre_Occurrences", axis=1, inplace=True)
previous_application.head()

,SK_ID_CURR,AMT_APPLICATION,AMT_CREDIT,NAME_CLIENT_TYPE
0,100001,24835.50,23787.00,Refreshed
1,100002,179055.00,179055.00,New
2,100003,435436.50,484191.00,Refreshed
3,100004,24282.00,20106.00,New
4,100005,22308.75,20076.75,New


## installments_payments

In [214]:
installments_payments

,SK_ID_CURR,AMT_PAYMENT
0,161674,6948.360
1,151639,1716.525
2,193053,25425.000
3,199697,24350.130
4,167756,2160.585
...,...,...
13605396,428057,NaN
13605397,414406,NaN
13605398,402199,NaN
13605399,409297,NaN


In [215]:

if installments_payments["AMT_PAYMENT"].isna().sum()/installments_payments.shape[0]*100 > 35: installments_payments.drop("AMT_PAYMENT", axis=1, inplace=True)
else : installments_payments.fillna({"AMT_PAYMENT": installments_payments["AMT_PAYMENT"].median()}, inplace=True)

In [216]:
installments_payments

,SK_ID_CURR,AMT_PAYMENT
0,161674,6948.360
1,151639,1716.525
2,193053,25425.000
3,199697,24350.130
4,167756,2160.585
...,...,...
13605396,428057,8125.515
13605397,414406,8125.515
13605398,402199,8125.515
13605399,409297,8125.515


In [217]:
installments_payments = installments_payments.groupby('SK_ID_CURR')['AMT_PAYMENT'].mean().reset_index(name='AMT_PAYMENT')

installments_payments



,SK_ID_CURR,AMT_PAYMENT
0,100001,5885.132143
1,100002,11559.247105
2,100003,64754.586000
3,100004,7096.155000
4,100005,6240.205000
...,...,...
339582,456251,7492.924286
339583,456252,10069.867500
339584,456253,4115.915357
339585,456254,10239.832895


## MERGING

In [218]:
ids = application["SK_ID_CURR"]

In [219]:
bureau = bureau[bureau["SK_ID_CURR"].isin(ids)]
bureau.reset_index(drop=True, inplace=True)
bureau

,SK_ID_CURR,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE
0,100002,108131.945625,30722.625000,3998.570625,0.0,Consumer credit
1,100003,254350.125000,0.000000,202500.000000,0.0,Consumer credit
2,100004,94518.900000,0.000000,0.000000,0.0,Consumer credit
3,100007,146250.000000,0.000000,0.000000,0.0,Consumer credit
4,100008,156148.500000,80019.000000,0.000000,0.0,Consumer credit
...,...,...,...,...,...,...
262338,456247,471405.681818,199399.090909,0.000000,0.0,Consumer credit
262339,456249,284142.973846,12543.923077,0.000000,0.0,Consumer credit
262340,456253,990000.000000,448958.250000,0.000000,0.0,Consumer credit
262341,456254,45000.000000,0.000000,0.000000,0.0,Consumer credit


In [220]:
POS_CASH_balance = POS_CASH_balance[POS_CASH_balance["SK_ID_CURR"].isin(ids)]
POS_CASH_balance.reset_index(drop=True, inplace=True)
POS_CASH_balance

,SK_ID_CURR,CNT_INSTALMENT_FUTURE
0,100002,15.000000
1,100003,5.785714
2,100004,2.250000
3,100006,8.571429
4,100007,8.969697
...,...,...
288108,456251,4.666667
288109,456252,3.000000
288110,456253,2.000000
288111,456254,10.350000


In [221]:
credit_card_balance = credit_card_balance[credit_card_balance["SK_ID_CURR"].isin(ids)]
credit_card_balance.reset_index(drop=True, inplace=True)
credit_card_balance

,SK_ID_CURR,AMT_BALANCE,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT
0,100006.0,0.000000,0.000000,2702.700000
1,100011.0,54482.111149,2432.432432,4843.064189
2,100021.0,0.000000,0.000000,2702.700000
3,100023.0,0.000000,0.000000,2702.700000
4,100036.0,0.000000,0.000000,2702.700000
...,...,...,...,...
86452,456242.0,148232.328750,39849.783750,19917.995625
86453,456244.0,131834.730732,26842.388049,32720.544878
86454,456246.0,13136.731875,15199.256250,16768.828125
86455,456247.0,23216.396211,2149.506474,4883.755263


In [222]:
previous_application = previous_application[previous_application["SK_ID_CURR"].isin(ids)]
previous_application.reset_index(drop=True, inplace=True)
previous_application

,SK_ID_CURR,AMT_APPLICATION,AMT_CREDIT,NAME_CLIENT_TYPE
0,100002,179055.000,179055.00,New
1,100003,435436.500,484191.00,Refreshed
2,100004,24282.000,20106.00,New
3,100006,272203.260,291695.50,Repeater
4,100007,150530.250,166638.75,Repeater
...,...,...,...,...
289707,456251,40455.000,40455.00,New
289708,456252,57595.500,56821.50,New
289709,456253,24162.750,20625.75,Refreshed
289710,456254,121317.750,134439.75,New


In [223]:
installments_payments = installments_payments[installments_payments["SK_ID_CURR"].isin(ids)]
installments_payments.reset_index(drop=True, inplace=True)
installments_payments

,SK_ID_CURR,AMT_PAYMENT
0,100002,11559.247105
1,100003,64754.586000
2,100004,7096.155000
3,100006,62947.088438
4,100007,12214.060227
...,...,...
290292,456251,7492.924286
290293,456252,10069.867500
290294,456253,4115.915357
290295,456254,10239.832895


In [224]:
data = pd.concat([application,bureau, POS_CASH_balance, credit_card_balance, previous_application, installments_payments], axis=1)

In [225]:
temp1 = application.merge(bureau, on='SK_ID_CURR', how='left')
temp2 = temp1.merge(POS_CASH_balance, on='SK_ID_CURR', how='left')
temp3 = temp2.merge(credit_card_balance, on='SK_ID_CURR', how='left')
temp4 = temp3.merge(previous_application, on='SK_ID_CURR', how='left')
data = temp4.merge(installments_payments, on='SK_ID_CURR', how='left')
data

,SK_ID_CURR,TARGET,date_mensuelle,NAME_CONTRACT_TYPE,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,CNT_INSTALMENT_FUTURE,AMT_BALANCE,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_APPLICATION,AMT_CREDIT_y,NAME_CLIENT_TYPE,AMT_PAYMENT
0,333721,0,2013-01-01,Cash loans,292500.0,1102500.0,32364.0,Working,Secondary / secondary special,Married,...,0.0,Consumer credit,5.894737,NaN,NaN,NaN,84628.500,87904.5000,Repeater,16194.380625
1,250254,0,2013-01-01,Cash loans,180000.0,270000.0,10179.0,Working,Higher education,Married,...,0.0,Consumer credit,11.112676,NaN,NaN,NaN,48510.900,50020.6500,Repeater,5617.057826
2,265071,0,2013-01-01,Cash loans,67500.0,80865.0,7546.5,Working,Secondary / secondary special,Married,...,0.0,Consumer credit,6.636364,NaN,NaN,NaN,42750.000,47263.5000,New,4260.588750
3,227569,0,2013-01-01,Cash loans,157500.0,1078200.0,38331.0,Working,Secondary / secondary special,Separated,...,0.0,Consumer credit,5.000000,NaN,NaN,NaN,34596.000,38943.0000,New,4301.827500
4,212616,0,2013-01-01,Cash loans,112500.0,225000.0,15165.0,Working,Secondary / secondary special,Civil marriage,...,0.0,Consumer credit,6.000000,NaN,NaN,NaN,71032.500,68175.0000,New,7946.006250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306102,150198,0,2020-12-01,Cash loans,148500.0,669600.0,32341.5,Pensioner,Secondary / secondary special,Married,...,0.0,Consumer credit,3.000000,NaN,NaN,NaN,76680.000,72427.5000,New,16484.292000
306103,377351,0,2020-12-01,Cash loans,90000.0,312768.0,22374.0,Commercial associate,Secondary / secondary special,Separated,...,0.0,Consumer credit,3.900000,NaN,NaN,NaN,73427.625,75255.1875,Repeater,12434.893393
306104,365430,0,2020-12-01,Cash loans,157500.0,1724220.0,45612.0,Commercial associate,Incomplete higher,Married,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306105,342888,0,2020-12-01,Cash loans,63000.0,177768.0,9216.0,Pensioner,Secondary / secondary special,Civil marriage,...,0.0,Consumer credit,15.333333,NaN,NaN,NaN,72031.500,87246.0000,New,5179.250000


In [226]:
for col in data.columns:
    print(col,":", data[col].isna().sum()/data.shape[0]*100)

SK_ID_CURR : 0.0
TARGET : 0.0
date_mensuelle : 0.0
NAME_CONTRACT_TYPE : 0.0
AMT_INCOME_TOTAL : 0.0
AMT_CREDIT_x : 0.0
AMT_ANNUITY : 0.0
NAME_INCOME_TYPE : 0.0
NAME_EDUCATION_TYPE : 0.0
NAME_FAMILY_STATUS : 0.0
NAME_HOUSING_TYPE : 0.0
DAYS_EMPLOYED : 0.0
ORGANIZATION_TYPE : 0.0
EXT_SOURCE_2 : 0.0
EXT_SOURCE_3 : 0.0
AMT_CREDIT_SUM : 14.296961519991374
AMT_CREDIT_SUM_DEBT : 14.296961519991374
AMT_CREDIT_SUM_LIMIT : 14.296961519991374
AMT_CREDIT_SUM_OVERDUE : 14.296961519991374
CREDIT_TYPE : 14.296961519991374
CNT_INSTALMENT_FUTURE : 5.878336660056777
AMT_BALANCE : 71.75595461717634
AMT_DRAWINGS_CURRENT : 71.75595461717634
AMT_PAYMENT_CURRENT : 71.75595461717634
AMT_APPLICATION : 5.355970297967704
AMT_CREDIT_y : 5.355970297967704
NAME_CLIENT_TYPE : 5.355970297967704
AMT_PAYMENT : 5.16486065330097


In [227]:
data_target_1 = data[data["TARGET"]==1]
for col in data_target_1.columns:
    print(col,":", data_target_1[col].isna().sum()/data_target_1.shape[0]*100)

SK_ID_CURR : 0.0
TARGET : 0.0
date_mensuelle : 0.0
NAME_CONTRACT_TYPE : 0.0
AMT_INCOME_TOTAL : 0.0
AMT_CREDIT_x : 0.0
AMT_ANNUITY : 0.0
NAME_INCOME_TYPE : 0.0
NAME_EDUCATION_TYPE : 0.0
NAME_FAMILY_STATUS : 0.0
NAME_HOUSING_TYPE : 0.0
DAYS_EMPLOYED : 0.0
ORGANIZATION_TYPE : 0.0
EXT_SOURCE_2 : 0.0
EXT_SOURCE_3 : 0.0
AMT_CREDIT_SUM : 17.936379508580675
AMT_CREDIT_SUM_DEBT : 17.936379508580675
AMT_CREDIT_SUM_LIMIT : 17.936379508580675
AMT_CREDIT_SUM_OVERDUE : 17.936379508580675
CREDIT_TYPE : 17.936379508580675
CNT_INSTALMENT_FUTURE : 4.837542051696972
AMT_BALANCE : 69.74407017842694
AMT_DRAWINGS_CURRENT : 69.74407017842694
AMT_PAYMENT_CURRENT : 69.74407017842694
AMT_APPLICATION : 3.9390197163905807
AMT_CREDIT_y : 3.9390197163905807
NAME_CLIENT_TYPE : 3.9390197163905807
AMT_PAYMENT : 3.815526125282119


In [228]:
for col in data_target_1.select_dtypes(include=np.number).columns:
    print(col,":", data_target_1[col].isna().sum()/data_target_1.shape[0]*100)
    if data_target_1[col].isna().sum()/data_target_1.shape[0]*100 > 35: data_target_1.drop(col, axis=1, inplace=True)
    else : data_target_1.fillna({col: data_target_1[col].median()}, inplace=True)

SK_ID_CURR : 0.0
TARGET : 0.0
AMT_INCOME_TOTAL : 0.0
AMT_CREDIT_x : 0.0
AMT_ANNUITY : 0.0
DAYS_EMPLOYED : 0.0
EXT_SOURCE_2 : 0.0
EXT_SOURCE_3 : 0.0
AMT_CREDIT_SUM : 17.936379508580675
AMT_CREDIT_SUM_DEBT : 17.936379508580675
AMT_CREDIT_SUM_LIMIT : 17.936379508580675
AMT_CREDIT_SUM_OVERDUE : 17.936379508580675
CNT_INSTALMENT_FUTURE : 4.837542051696972
AMT_BALANCE : 69.74407017842694
AMT_DRAWINGS_CURRENT : 69.74407017842694
AMT_PAYMENT_CURRENT : 69.74407017842694
AMT_APPLICATION : 3.9390197163905807
AMT_CREDIT_y : 3.9390197163905807
AMT_PAYMENT : 3.815526125282119


/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/2131185450.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else : data_target_1.fillna({col: data_target_1[col].median()}, inplace=True)
/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/2131185450.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else : data_target_1.fillna({col: data_target_1[col].median()}, inplace=True)
/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/2131185450.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [229]:
for col in data_target_1.select_dtypes(include="object").columns:
    print(col,":", data_target_1[col].isna().sum()/data_target_1.shape[0]*100)
    if data_target_1[col].isna().sum()/data_target_1.shape[0]*100 > 35: data_target_1.drop(col, axis=1, inplace=True)
    else : data_target_1.fillna({col: data_target_1[col].mode()}, inplace=True)

date_mensuelle : 0.0
NAME_CONTRACT_TYPE : 0.0
NAME_INCOME_TYPE : 0.0
NAME_EDUCATION_TYPE : 0.0
NAME_FAMILY_STATUS : 0.0
NAME_HOUSING_TYPE : 0.0
ORGANIZATION_TYPE : 0.0
CREDIT_TYPE : 17.936379508580675
NAME_CLIENT_TYPE : 3.9390197163905807


/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/1164021784.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else : data_target_1.fillna({col: data_target_1[col].mode()}, inplace=True)


In [230]:
data_target_0 = data[data["TARGET"]==0]
for col in data_target_0.columns:
    print(col,":", data_target_0[col].isna().sum()/data_target_0.shape[0]*100)

SK_ID_CURR : 0.0
TARGET : 0.0
date_mensuelle : 0.0
NAME_CONTRACT_TYPE : 0.0
AMT_INCOME_TOTAL : 0.0
AMT_CREDIT_x : 0.0
AMT_ANNUITY : 0.0
NAME_INCOME_TYPE : 0.0
NAME_EDUCATION_TYPE : 0.0
NAME_FAMILY_STATUS : 0.0
NAME_HOUSING_TYPE : 0.0
DAYS_EMPLOYED : 0.0
ORGANIZATION_TYPE : 0.0
EXT_SOURCE_2 : 0.0
EXT_SOURCE_3 : 0.0
AMT_CREDIT_SUM : 13.994565217391305
AMT_CREDIT_SUM_DEBT : 13.994565217391305
AMT_CREDIT_SUM_LIMIT : 13.994565217391305
AMT_CREDIT_SUM_OVERDUE : 13.994565217391305
CREDIT_TYPE : 13.994565217391305
CNT_INSTALMENT_FUTURE : 5.96481544384058
AMT_BALANCE : 71.92312047101449
AMT_DRAWINGS_CURRENT : 71.92312047101449
AMT_PAYMENT_CURRENT : 71.92312047101449
AMT_APPLICATION : 5.473703577898551
AMT_CREDIT_y : 5.473703577898551
NAME_CLIENT_TYPE : 5.473703577898551
AMT_PAYMENT : 5.276975769927536


In [231]:
for col in data_target_0.select_dtypes(include="object").columns:
    print(col,":", data_target_0[col].isna().sum()/data_target_0.shape[0]*100)
    if data_target_0[col].isna().sum()/data_target_0.shape[0]*100 > 35: data_target_0.drop(col, axis=1, inplace=True)
    else : data_target_0.fillna({col: data_target_0[col].mode()}, inplace=True)

for col in data_target_0.select_dtypes(include=np.number).columns:
    print(col,":", data_target_0[col].isna().sum()/data_target_0.shape[0]*100)
    if data_target_0[col].isna().sum()/data_target_0.shape[0]*100 > 35: data_target_0.drop(col, axis=1, inplace=True)
    else : data_target_0.fillna({col: data_target_0[col].mode()}, inplace=True)

date_mensuelle : 0.0


/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/997673985.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else : data_target_0.fillna({col: data_target_0[col].mode()}, inplace=True)


NAME_CONTRACT_TYPE : 0.0
NAME_INCOME_TYPE : 0.0
NAME_EDUCATION_TYPE : 0.0
NAME_FAMILY_STATUS : 0.0
NAME_HOUSING_TYPE : 0.0
ORGANIZATION_TYPE : 0.0
CREDIT_TYPE : 13.994565217391305
NAME_CLIENT_TYPE : 5.473703577898551
SK_ID_CURR : 0.0
TARGET : 0.0
AMT_INCOME_TOTAL : 0.0
AMT_CREDIT_x : 0.0
AMT_ANNUITY : 0.0
DAYS_EMPLOYED : 0.0
EXT_SOURCE_2 : 0.0


/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/997673985.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else : data_target_0.fillna({col: data_target_0[col].mode()}, inplace=True)
/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/997673985.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else : data_target_0.fillna({col: data_target_0[col].mode()}, inplace=True)
/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/997673985.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

EXT_SOURCE_3 : 0.0
AMT_CREDIT_SUM : 13.994565217391305
AMT_CREDIT_SUM_DEBT : 13.994565217391305
AMT_CREDIT_SUM_LIMIT : 13.994565217391305
AMT_CREDIT_SUM_OVERDUE : 13.994565217391305
CNT_INSTALMENT_FUTURE : 5.96481544384058
AMT_BALANCE : 71.92312047101449
AMT_DRAWINGS_CURRENT : 71.92312047101449
AMT_PAYMENT_CURRENT : 71.92312047101449


/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/997673985.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if data_target_0[col].isna().sum()/data_target_0.shape[0]*100 > 35: data_target_0.drop(col, axis=1, inplace=True)
/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/997673985.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if data_target_0[col].isna().sum()/data_target_0.shape[0]*100 > 35: data_target_0.drop(col, axis=1, inplace=True)
/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_22037/997673985.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy 

AMT_APPLICATION : 5.473703577898551
AMT_CREDIT_y : 5.473703577898551
AMT_PAYMENT : 5.276975769927536


In [232]:
data_final = pd.concat([data_target_1, data_target_0], axis=0)

In [234]:
data_final["date_mensuelle"] = pd.to_datetime(data_final["date_mensuelle"])

In [237]:
data_final.sort_values(by="date_mensuelle", ascending=True, inplace=True)
data_final.head(30)

,SK_ID_CURR,TARGET,date_mensuelle,NAME_CONTRACT_TYPE,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,CNT_INSTALMENT_FUTURE,AMT_APPLICATION,AMT_CREDIT_y,NAME_CLIENT_TYPE,AMT_PAYMENT
848,341177,1,2013-01-01,Cash loans,99000.0,354276.0,28120.5,Working,Secondary / secondary special,Civil marriage,...,5.249025e+04,0.000000e+00,0.000000,0.0,Consumer credit,5.000000,65316.150000,69522.300000,Repeater,4164.552692
2206,410860,0,2013-01-01,Cash loans,157500.0,225000.0,9909.0,Pensioner,Higher education,Married,...,1.536994e+05,3.730875e+03,0.000000,0.0,Consumer credit,5.000000,167314.500000,179275.500000,New,19716.777000
2207,125382,0,2013-01-01,Cash loans,76500.0,808650.0,23773.5,Pensioner,Secondary / secondary special,Married,...,1.922505e+05,4.759200e+04,0.000000,0.0,Consumer credit,6.785714,170644.500000,168273.000000,New,12352.523824
2208,416718,0,2013-01-01,Cash loans,157500.0,315000.0,11673.0,Working,Secondary / secondary special,Married,...,4.181973e+05,2.029303e+05,0.000000,0.0,Consumer credit,8.707692,97378.875000,118605.937500,Repeater,27195.907846
2209,280621,0,2013-01-01,Cash loans,90000.0,1078200.0,31522.5,Working,Secondary / secondary special,Married,...,6.907950e+04,0.000000e+00,0.000000,0.0,Credit card,4.222222,76363.137000,92751.300000,Repeater,5820.422344
2210,294363,0,2013-01-01,Cash loans,180000.0,1216201.5,35559.0,Commercial associate,Secondary / secondary special,Single / not married,...,1.508481e+05,0.000000e+00,0.000000,0.0,Consumer credit,5.820513,63136.125000,161503.312500,Repeater,45310.671964
2211,377460,0,2013-01-01,Cash loans,135000.0,675000.0,21906.0,Working,Secondary / secondary special,Separated,...,1.875000e+05,1.373460e+05,252.000000,0.0,Credit card,6.904762,92634.428571,81013.500000,Repeater,7062.249079
2212,121625,0,2013-01-01,Cash loans,157500.0,428854.5,20119.5,Working,Incomplete higher,Civil marriage,...,4.003200e+04,0.000000e+00,0.000000,0.0,Consumer credit,0.909091,225000.000000,225000.000000,Refreshed,5958.225000
2213,278879,0,2013-01-01,Cash loans,202500.0,1762110.0,48586.5,Commercial associate,Secondary / secondary special,Married,...,2.792592e+05,0.000000e+00,0.000000,0.0,Consumer credit,7.250000,495000.000000,445500.000000,New,77269.507500
2214,320803,0,2013-01-01,Cash loans,225000.0,646920.0,20997.0,Working,Higher education,Married,...,2.013744e+05,4.657056e+04,8053.818750,0.0,Consumer credit,31.435897,203405.062500,211312.687500,Repeater,8654.251711


In [238]:
data_final.to_csv("data/data_variables_metiers.csv", index=False)